In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import numpy as np
import pandas as pd
import re
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
def needpara(x):
    if '[[**##**]]' not in str(x):
        return x
    zz=x.split('. ')
    if '[[**##**]]' in zz:
        index=zz.index('[[**##**]]')
        first=max(0,index-2)
        return ''.join(zz[first:index+2])
    elif ' [[**##**]]' in zz:
        index=zz.index(' [[**##**]]')
#         if len(zz[index-1])>30:
#             return ''.join(zz[index-1])
#         else:
        first=max(0,index-2)
        return ''.join(zz[first:index+2])
    elif '[[**##**]] ' in zz:
        index=zz.index('[[**##**]] ')
#         if len(zz[index-1])>30:
#             return ''.join(zz[index-1])
#         else:
        first=max(0,index-2)
        return ''.join(zz[first:index+2])
    else:
        for t in zz:
            if '[[**##**]]' in t:
                index=zz.index(t)
                first=max(0,index-2)
                return ''.join(zz[first:index+2])#''.join(t.split('[[**##**]]'))
            
            
'获取关键句函数'
def digest(text):
    backup = text[:]
    text = text.replace('al.', '').split('. ')
    t=''
    pre_text=[]
    len_text=len(text)-1
    add=True
    pre=''
    while len_text>=0:
        index=text[len_text]
        index+=pre
        if len(index.split(' '))<=3 :
            add=False
            pre=index+pre
        else:
            add=True
            pre=''
        if add:
            pre_text.append(index)
        len_text-=1
    if len(pre_text)==0:
        pre_text=text
    pre_text.reverse()
    for index in pre_text:
        if index.find('[**##**]') != -1:
            index = re.sub(r'[\[|,]+\*\*\#\#\*\*[\]|,]+','',index)
            index+='. '
            t+=index
    return t

In [ ]:
def get_label(x):
    if len(str(x))<11:
        return 1
    return 0

def get_label_title(x):
    if len(str(x))<50:
        return 1
    return 0

In [ ]:
press2 = pd.read_csv('./data/train_pre.csv')#train_release.csv
press2=press2[~press2['description_id'].isnull()]
press1 = pd.read_csv('./data/test_pre2.csv')#validation
paper = pd.read_csv('./data/candidate_paper_pre.csv')# candidate_paper_for_wsdm2020.csv

In [ ]:
paper['all_info']=paper['title']+paper['abstract']
paper['all_info_pre']=paper['title_pro']+paper['abstract_pre']

In [ ]:
print('press2 shape',press2.shape)
press2=pd.merge(press2,paper,on='paper_id',how='left')
print('press2 shape',press2.shape)
press2=press2.drop_duplicates(['description_text']).reset_index(drop=True)
print('press2 shape',press2.shape)
# press2=press2[press2.year.notnull()].reset_index(drop=True)

In [ ]:
paper=paper[paper.journal=='no-content'].reset_index(drop=True)
print(paper.shape)

In [ ]:
press  = pd.concat([press1 ,press2]).reset_index(drop=True)
press['description_text']=press['description_text'].fillna('')
press2['description_text']=press2['description_text'].fillna('')
paper['title']=paper['title'].fillna('')
paper['abstract']=paper['abstract'].fillna('')

article = list(press['description_text'])
article.extend(list(paper['title']))
article.extend(list(paper['abstract']))

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1, 1), min_df=2, max_df=0.5, sublinear_tf=True,stop_words='english')#
vectorizer.fit(article)
yy = vectorizer.transform(paper['title'])
xx = vectorizer.transform(press2['description_text'])

yy1 = vectorizer.transform(paper['abstract'])
xx1 = xx

In [ ]:
# del score
import gc
gc.collect()

In [ ]:
article = list(press['key_text_pre'].fillna(''))
article.extend(list(paper['title_pro'].fillna('')))
article.extend(list(paper['abstract_pre'].fillna('')))

vectorizer = TfidfVectorizer(ngram_range=(1, 1), min_df=2, max_df=0.5, sublinear_tf=True,stop_words='english')#
vectorizer.fit(article)
yy2 = vectorizer.transform(paper['title_pro'].fillna(''))
xx2 = vectorizer.transform(press2['key_text_pre'].fillna(''))

yy3 = vectorizer.transform(paper['abstract_pre'].fillna(''))
xx3 = xx2

In [ ]:
paper_id=list(paper.paper_id)

In [ ]:
max_epoch=len(press2)//1000

In [ ]:
result=[]
batch=1000
from tqdm import tqdm 
for epoch in tqdm(range(0,max_epoch)):
    first=np.dot(xx[epoch*batch:batch*(epoch+1)],yy.T)
    score=np.array(list(np.array(first.todense())[0:batch]))
    del first 
    gc.collect()
    
    first=np.dot(xx1[epoch*batch:batch*(epoch+1)],yy1.T)
    score1=np.array(list(np.array(first.todense())[0:batch]))
    del first 
    gc.collect()
    
    first=np.dot(xx2[epoch*batch:batch*(epoch+1)],yy2.T)
    score2=np.array(list(np.array(first.todense())[0:batch]))
    del first 
    gc.collect()
    
    first=np.dot(xx3[epoch*batch:batch*(epoch+1)],yy3.T)
    score3=np.array(list(np.array(first.todense())[0:batch]))
    del first 
    gc.collect()
   
   
    all_score=score
    for i in range(batch):
        all_score[i]=0.65*(1*score[i]+3*score1[i])+0.35*(1*score2[i]+3*score3[i])
        zz=list(np.argsort(-np.array(all_score[i]))[0:100])
#         for k in range(100):
#             zz[k]=paper_id[zz[k]]
#         result.append(zz)
        for k in range(100):
            get_score=np.array(all_score[i])[zz[k]]
            zz[k]=paper_id[zz[k]]
            result.append([zz[k],get_score])

In [ ]:
there1=max_epoch*1000
batch=xx.shape[0]-there1
first=np.dot(xx[there1:batch+there1],yy.T)
score=np.array(list(np.array(first.todense())[0:batch]))
first=np.dot(xx1[there1:batch+there1],yy1.T)
score1=np.array(list(np.array(first.todense())[0:batch]))
first=np.dot(xx2[there1:there1+batch],yy2.T)
score2=np.array(list(np.array(first.todense())[0:batch]))
first=np.dot(xx3[there1:there1+batch],yy3.T)
score3=np.array(list(np.array(first.todense())[0:batch]))


In [ ]:
all_score=score
for i in range(batch):
    all_score[i]=0.65*(1*score[i]+3*score1[i])+0.35*(1*score2[i]+3*score3[i])
    zz=list(np.argsort(-np.array(all_score[i]))[0:100])
#     for k in range(100):
#         zz[k]=paper_id[zz[k]]
#     result.append(zz)

    for k in range(100):
        get_score=np.array(all_score[i])[zz[k]]
        zz[k]=paper_id[zz[k]]
        result.append([zz[k],get_score])

In [ ]:
len(result)

In [ ]:
describe_id=list(press2['description_id'])
idd=[]
for t in describe_id:
    idd.extend([t]*100)

In [ ]:
len(idd)

In [ ]:
feature=pd.DataFrame(result)
feature.columns=['paper_id','score']

In [ ]:
feature['description_id']=idd

In [ ]:
press2['label']=1

In [ ]:
feature=pd.merge(feature,paper,on='paper_id',how='left')

In [ ]:
feature=pd.merge(feature,press2[['description_id','description_text','key_text','key_text_pre']],on='description_id',how='left')

In [ ]:
data=pd.merge(feature,press2[['description_id','paper_id','label']],on=['description_id','paper_id'],how='left')#feature#

In [ ]:
data['label']=data['label'].fillna(0)

In [ ]:
data=data.drop_duplicates(subset=['description_id','paper_id'], keep='first', inplace=False)

In [ ]:
data=data.reset_index(drop=True)

In [ ]:
data=data.sort_values(by='description_id').reset_index(drop=True)

In [ ]:
data_p=data[['description_id','paper_id','description_text','key_text_pre','abstract','abstract_pre','title','keywords','title_pro','journal','year','label']]

In [ ]:
data_p['score']=data['score']

In [ ]:
data_p.to_csv('train_feature/train2yu_100.csv',index=None)